In [1]:
# library import
from math import *
from sympy import *
import numpy as np
from graphics import *
import matplotlib.pyplot as plt

# parameters (units: mm)
l1 = 83 # first leg linkage (femur)
l2 = 83 # second leg linkage (tibia)

# inverse kinematics solver function
def inverseKinematics(x, y, z):

    th1 = atan(y/z)
    r1 = sqrt( (z**2) + (y**2) )
    r3 = sqrt( (x**2) + (y**2) + (z**2) )
    
    ph3 = acos( (l1**2 + l2**2 - r3**2) / (2 * l1 * l2) )
    ph1 = atan( x / sqrt(z**2 + y**2) )
    ph2 = acos( (l2**2 + r3**2 - l1**2) / (2 * l2 * r3) )

    th2 = ph3 + ph2 - ph1 - 0.5*pi
    th3 = ph3 - th2
    
    return th1, th2, th3

def inverseKPolar(R, th, x):

    th = th * pi / 180
    r1 = ( R**2 - x**2 )**0.5
    y = r1 * sin(th)
    z = r1 * cos(th)

    th1, th2, th3 = inverseKinematics(x, y, z)
    return th1, th2, th3

# output the ik sequence in a format that can be ctr-c / ctr-v into the ESP32 code
def printSequence(lx, ly, lz, rx, ry, rz, torso, larm, rarm):
    for i in range(len(lx)):

        # left leg
        lth1, lth2, lth3 = inverseKinematics(lx[i], ly[i], lz[i])
        lth1 = round(float(lth1 * 180 / pi), 1)
        lth2 = round(90-float(lth2 * 180 / pi), 1)
        lth3 = round(90-float(lth3 * 180 / pi), 1)

        # right leg
        rth1, rth2, rth3 = inverseKinematics(rx[i], ry[i], rz[i])
        rth1 = round(float(rth1 * 180 / pi), 1)
        rth2 = round(90-float(rth2 * 180 / pi), 1)
        rth3 = round(90-float(rth3 * 180 / pi), 1)
        tso = (round(float(torso[i]), 1))
        la = 90 + (round(float(larm[i]), 1))
        ra = 90 + (round(float(rarm[i]), 1))

        ra = -1 * ra
        rth1 = -1 * rth1
        rth2 = -1 * (rth2)
        rth3 =  1 * (rth3)

        la =  1 * la
        lth1 =  1 * lth1
        lth2 =  1 * (lth2)
        lth3 = -1 * (lth3)
        
        print(str(tso) + ", "
              + str(ra) + ", "
              + str(rth1) + ", " + str(rth2) + ", " + str(rth3) + ", " + str(rth1) + ", "
              + str(lth1) + ", " + str(lth2) + ", " + str(lth3) + ", " + str(lth1) + ", "
              + str(la) + ",")

def drawLeg(x,y,z):

    th1, th2, th3 = inverseKinematics(x,y,z)
    
    r1 = l1 * exp(I*(-1*th2))
    r2 = l2 * exp(I*(th3 + pi))

    ptO = Point(0,0)
    ptA = Point(re(r1), im(r1))
    ptB = Point(re(r1 + r2), im(r1 + r2))

    line1 = Line(ptO, ptA)
    line2 = Line(ptA, ptB)

    line1.setWidth(3)
    line2.setWidth(3)
    
    return line1,line2, ptB

def draw_grid(win, x_min, y_min, x_max, y_max, rows, cols, color="gray"):
    # Compute step sizes in logical units
    x_step = (x_max - x_min) / cols
    y_step = (y_max - y_min) / rows

    # Draw vertical lines
    for i in range(cols + 1):
        x = x_min + i * x_step
        line = Line(Point(x, y_min), Point(x, y_max))
        line.setOutline(color)
        line.draw(win)

    # Draw horizontal lines
    for j in range(rows + 1):
        y = y_min + j * y_step
        line = Line(Point(x_min, y), Point(x_max, y))
        line.setOutline(color)
        line.draw(win)

def animateSequence(ilx, ily, ilz, irx, iry, irz, wlx, wly, wlz, wrx, wry, wrz):

    # create graphical window interface
    win = GraphWin("Window", 640, 640)
    win.setCoords(-120, -280, 120, 5)
    draw_grid(win, -120,-280,120,5,20,20)
    
    pline1 = None
    pline2 = None
    pline3 = None
    pline4 = None

    win.getMouse()

    for i in range( len(ilx) ):

        line1, line2, ptL = drawLeg(ilx[i], ily[i], ilz[i])
        line3, line4, ptR = drawLeg(irx[i], iry[i], irz[i])
        line1.setOutline('red')
        line2.setOutline('red')
        line3.setOutline('blue')
        line4.setOutline('blue')
        line1.draw(win)
        line2.draw(win)
        line3.draw(win)
        line4.draw(win)
        ptL.draw(win)

        if pline1 != None:
            pline1.undraw()
            pline2.undraw()
            pline3.undraw()
            pline4.undraw()

        pline1 = line1
        pline2 = line2
        pline3 = line3
        pline4 = line4

        update(10)

    for i in range( int(len(wlx) / 4) , len(wlx) ):

        line1, line2, ptL = drawLeg(wlx[i], wly[i], wlz[i])
        line3, line4, ptR = drawLeg(walkRX[i], wry[i], wrz[i])
        line1.setOutline('red')
        line2.setOutline('red')
        line3.setOutline('blue')
        line4.setOutline('blue')
        line1.draw(win)
        line2.draw(win)
        line3.draw(win)
        line4.draw(win)
        ptL.draw(win)
    
        if pline1 != None:
            pline1.undraw()
            pline2.undraw()
            pline3.undraw()
            pline4.undraw()
    
        pline1 = line1
        pline2 = line2
        pline3 = line3
        pline4 = line4

        update(10)

    for x in range(5):
        for i in range( len(walkLX) ):
        
            line1, line2, ptL = drawLeg(wlx[i], wly[i], wlz[i])
            line3, line4, ptR = drawLeg(wrx[i], wry[i], wrz[i])
            line1.setOutline('red')
            line2.setOutline('red')
            line3.setOutline('blue')
            line4.setOutline('blue')
            line1.draw(win)
            line2.draw(win)
            line3.draw(win)
            line4.draw(win)
            ptL.draw(win)
        
            if pline1 != None:
                pline1.undraw()
                pline2.undraw()
                pline3.undraw()
                pline4.undraw()
        
            pline1 = line1
            pline2 = line2
            pline3 = line3
            pline4 = line4
    
            update(10)

    win.close()

def printStep(list):
    for x in list:
        print(str(x) + ",", end="")
    print("")
    return len(list)

# Forwards/Backwards Sequence Generation

First start with defining some parameters and a mathematical model for the trajectory of the foot in XYZ coordinates.  We have an initial step sequence and a continuous step sequence; the initial sequence is only run once and leads into the continuous sequence that repeats until a new serial command from the ESP32 interrupts the loop

In [2]:
# z0: initial vertical height
# nZ: maximum negative vertical displacement (towards body from z0)
# pZ: maximum positive vertical displacement (away from body)
# y0: initial foot side displacement from center
# dY: left/right foot amplitude
# x0: initial foot front/back displacement from center (COM compensation)
# dX: forward/backward foot amplitude
# tD: torso swivel joint amplitude (multiply by 2/3 if using 270 deg servo)
# dA: shoulder arm joint amplitude (multiply by 2/3 if using 270 deg servo)

# latest stable: 125 30 10 5 10 10 25 20 20
# new stable: 125 30 15 5 20 20 25 10 20
# 125 40 15 5 25 20 25 10 20

z0 = 125
nZ = 30
pZ = 15
y0 = 5
dY = 25
x0 = 30 # 20
dX = 25
tD = 10 # 15
dA = 20

# Initial sequence steps
initLX = [x0, x0 + dX/6, x0 + dX/3, x0 + dX/2, x0 + dX*2/3, x0 + dX*5/6]
initRX = [x0, x0 - dX/6, x0 - dX/3, x0 - dX/2, x0 - dX*2/3, x0 - dX*5/6]

initRY = [y0, y0 - dY*sin(pi/6), y0 - dY*sin(pi/3), y0 - dY, y0 - dY*sin(pi/3), y0 - dY*sin(pi/6)] 
initLY = [y0, y0 + dY*sin(pi/6), y0 + dY*sin(pi/3), y0 + dY, y0 + dY*sin(pi/3), y0 + dY*sin(pi/6)]

initLZ = [z0, z0 - nZ*sin(pi/6), z0 - nZ*sin(pi/3), z0 - nZ, z0 - nZ*sin(pi/3), z0 - nZ*sin(pi/6)]
initRZ = [z0, z0 + pZ*sin(pi/6), z0 + pZ*sin(pi/3), z0 + pZ, z0 + pZ*sin(pi/3), z0 + pZ*sin(pi/6)]

initT =  [0, tD*sin(pi/12), tD*sin(pi/6), tD*sin(pi/4), tD*sin(pi/3), tD*sin(5*pi/12)]
initRA = [0, -dA*sin(pi/12), -dA*sin(pi/6), -dA*sin(pi/4), -dA*sin(pi/3), -dA*sin(5*pi/12)]
initLA = [0, dA*sin(pi/12), dA*sin(pi/6), dA*sin(pi/4), dA*sin(pi/3), dA*sin(5*pi/12)]

# Continuous sequence steps
walkLX = [x0, x0 + dX/3, x0 + dX*2/3, x0 + dX, x0 + dX*2/3, x0 + dX/3, x0, x0 - dX/3, x0 - dX*2/3, x0 - dX, x0 - dX*2/3, x0 - dX/3]
walkRX = [x0, x0 - dX/3, x0 - dX*2/3, x0 - dX, x0 - dX*2/3, x0 - dX/3, x0, x0 + dX/3, x0 + dX*2/3, x0 + dX, x0 + dX*2/3, x0 + dX/3]

walkRY = [y0 - dY, y0 - dY*cos(pi/6), y0 - dY*0.5, y0, y0 + dY*0.5, y0 + dY*cos(pi/6), 
          y0 + dY, y0 + dY*cos(pi/6), y0 + dY*0.5, y0, y0 - dY*0.5, y0 - dY*cos(pi/6)]
walkLY = [y0 + dY, y0 + dY*cos(pi/6), y0 + dY*0.5, y0, y0 - dY*0.5, y0 - dY*cos(pi/6), 
          y0 - dY, y0 - dY*cos(pi/6), y0 - dY*0.5, y0, y0 + dY*0.5, y0 + dY*cos(pi/6)]

walkLZ = [z0 - nZ, z0 - nZ*cos(pi/6), z0 - nZ*0.5, z0, z0 + pZ*0.5, z0 + pZ*cos(pi/6),
          z0 + pZ, z0 + pZ*cos(pi/6), z0 + pZ*0.5, z0, z0 - nZ*0.5, z0 - nZ*cos(pi/6)]
walkRZ = [z0 + pZ, z0 + pZ*cos(pi/6), z0 + pZ*0.5, z0, z0 - nZ*0.5, z0 - nZ*cos(pi/6),
          z0 - nZ, z0 - nZ*cos(pi/6), z0 - nZ*0.5, z0, z0 + pZ*0.5, z0 + pZ*cos(pi/6)]

walkT = [0, tD*sin(pi/6), tD*sin(pi/3), tD, tD*sin(pi/3), tD*sin(pi/6), 0, -tD*sin(pi/6), -tD*sin(pi/3), -tD, -tD*sin(pi/3), -tD*sin(pi/6)]
walkRA = [0, -dA*sin(pi/6), -dA*sin(pi/3), -dA, -dA*sin(pi/3), -dA*sin(pi/6), 0, dA*sin(pi/6), dA*sin(pi/3), dA, dA*sin(pi/3), dA*sin(pi/6)]
walkLA = [0, dA*sin(pi/6), dA*sin(pi/3), dA, dA*sin(pi/3), dA*sin(pi/6), 0, -dA*sin(pi/6), -dA*sin(pi/3), -dA, -dA*sin(pi/3), -dA*sin(pi/6)]

# Generate step sequences
print("Initial forwards walk sequence:")
printSequence(initLX, initLY, initLZ, initRX, initRY, initRZ, initT, initLA, initRA)
print("-----")

print("Continuous walk sequence:")
printSequence(walkLX, walkLY, walkLZ, walkRX, walkRY, walkRZ, walkT, walkLA, walkRA)
print("-----")

animateSequence(initLX, initLY, initLZ, initRX, initRY, initRZ, walkLX, walkLY, walkLZ, walkRX, walkRY, walkRZ)

Initial forwards walk sequence:
0.0, -90.0, -2.3, -52.7, 25.7, -2.3, 2.3, 52.7, -25.7, 2.3, 90.0,
2.6, -84.8, 3.2, -46.5, 24.4, 3.2, 9.0, 62.5, -28.4, 9.0, 95.2,
5.0, -80.0, 6.9, -40.9, 23.2, 6.9, 15.1, 69.2, -28.2, 15.1, 100.0,
7.1, -75.9, 8.1, -37.9, 23.8, 8.1, 17.5, 72.4, -26.2, 17.5, 104.1,
8.7, -72.7, 6.9, -38.2, 27.3, 6.9, 15.1, 71.7, -22.8, 15.1, 107.3,
9.7, -70.7, 3.2, -40.7, 32.8, 3.2, 9.0, 67.0, -17.9, 9.0, 109.3,
-----
Continuous walk sequence:
0.0, -90.0, 8.1, -41.4, 17.5, 8.1, 17.5, 67.9, -34.4, 17.5, 90.0,
5.0, -80.0, 6.9, -40.9, 23.2, 6.9, 15.1, 69.2, -28.2, 15.1, 100.0,
8.7, -72.7, 3.2, -42.3, 30.8, 3.2, 9.0, 66.1, -20.6, 9.0, 107.3,
10.0, -70.0, -2.3, -43.3, 38.8, -2.3, 2.3, 58.3, -10.9, 2.3, 110.0,
8.7, -72.7, -9.0, -54.3, 40.7, -9.0, -3.2, 51.4, -12.7, -3.2, 107.3,
5.0, -80.0, -15.1, -62.8, 38.9, -15.1, -6.9, 45.1, -14.3, -6.9, 100.0,
0.0, -90.0, -17.5, -67.9, 34.4, -17.5, -8.1, 41.4, -17.5, -8.1, 90.0,
-5.0, -100.0, -15.1, -69.2, 28.2, -15.1, -6.9, 40.9, -23.2, -6.9

In [4]:
# latest stable: 125 30 10 5 10 10 25 20 20
# new stable: 125 30 15 5 20 20 25 10 20

z0 = 125
nZ = 40
pZ = 15
y0 = 5
dY = 25
x0 = 20
dX = 25
tD = 10
dA = 20

# Initial sequence steps
initLX = [x0, x0 + dX/6, x0 + dX/3, x0 + dX/2, x0 + dX*2/3, x0 + dX*5/6]
initRX = [x0, x0 - dX/6, x0 - dX/3, x0 - dX/2, x0 - dX*2/3, x0 - dX*5/6]

initLY = [y0, y0 - dY*sin(pi/6), y0 - dY*sin(pi/3), y0 - dY, y0 - dY*sin(pi/3), y0 - dY*sin(pi/6)] 
initRY = [y0, y0 + dY*sin(pi/6), y0 + dY*sin(pi/3), y0 + dY, y0 + dY*sin(pi/3), y0 + dY*sin(pi/6)]

initRZ = [z0, z0 - nZ*sin(pi/6), z0 - nZ*sin(pi/3), z0 - nZ, z0 - nZ*sin(pi/3), z0 - nZ*sin(pi/6)]
initLZ = [z0, z0 + pZ*sin(pi/6), z0 + pZ*sin(pi/3), z0 + pZ, z0 + pZ*sin(pi/3), z0 + pZ*sin(pi/6)]

initT =  [0, tD*sin(pi/12), tD*sin(pi/6), tD*sin(pi/4), tD*sin(pi/3), tD*sin(5*pi/12)]
initLA = [0, dA*sin(pi/12), dA*sin(pi/6), dA*sin(pi/4), dA*sin(pi/3), dA*sin(5*pi/12)]
initRA = [0, -dA*sin(pi/12), -dA*sin(pi/6), -dA*sin(pi/4), -dA*sin(pi/3), -dA*sin(5*pi/12)]

# Continuous sequence steps
walkLX = [x0, x0 + dX/3, x0 + dX*2/3, x0 + dX, x0 + dX*2/3, x0 + dX/3, x0, x0 - dX/3, x0 - dX*2/3, x0 - dX, x0 - dX*2/3, x0 - dX/3]
walkRX = [x0, x0 - dX/3, x0 - dX*2/3, x0 - dX, x0 - dX*2/3, x0 - dX/3, x0, x0 + dX/3, x0 + dX*2/3, x0 + dX, x0 + dX*2/3, x0 + dX/3]

walkLY = [y0 - dY, y0 - dY*cos(pi/6), y0 - dY*0.5, y0, y0 + dY*0.5, y0 + dY*cos(pi/6), 
          y0 + dY, y0 + dY*cos(pi/6), y0 + dY*0.5, y0, y0 - dY*0.5, y0 - dY*cos(pi/6)]
walkRY = [y0 + dY, y0 + dY*cos(pi/6), y0 + dY*0.5, y0, y0 - dY*0.5, y0 - dY*cos(pi/6), 
          y0 - dY, y0 - dY*cos(pi/6), y0 - dY*0.5, y0, y0 + dY*0.5, y0 + dY*cos(pi/6)]

walkRZ = [z0 - nZ, z0 - nZ*cos(pi/6), z0 - nZ*0.5, z0, z0 + pZ*0.5, z0 + pZ*cos(pi/6),
          z0 + pZ, z0 + pZ*cos(pi/6), z0 + pZ*0.5, z0, z0 - nZ*0.5, z0 - nZ*cos(pi/6)]
walkLZ = [z0 + pZ, z0 + pZ*cos(pi/6), z0 + pZ*0.5, z0, z0 - nZ*0.5, z0 - nZ*cos(pi/6),
          z0 - nZ, z0 - nZ*cos(pi/6), z0 - nZ*0.5, z0, z0 + pZ*0.5, z0 + pZ*cos(pi/6)]

walkT = [0, tD*sin(pi/6), tD*sin(pi/3), tD, tD*sin(pi/3), tD*sin(pi/6), 0, -tD*sin(pi/6), -tD*sin(pi/3), -tD, -tD*sin(pi/3), -tD*sin(pi/6)]
walkLA = [0, dA*sin(pi/6), dA*sin(pi/3), dA, dA*sin(pi/3), dA*sin(pi/6), 0, -dA*sin(pi/6), -dA*sin(pi/3), -dA, -dA*sin(pi/3), -dA*sin(pi/6)]
walkRA = [0, -dA*sin(pi/6), -dA*sin(pi/3), -dA, -dA*sin(pi/3), -dA*sin(pi/6), 0, dA*sin(pi/6), dA*sin(pi/3), dA, dA*sin(pi/3), dA*sin(pi/6)]

# Generate step sequences
print("Initial backwards walk sequence:")
printSequence(initLX, initLY, initLZ, initRX, initRY, initRZ, initT, initLA, initRA)
print("-----")

print("Continuous walk sequence:")
printSequence(walkLX, walkLY, walkLZ, walkRX, walkRY, walkRZ, walkT, walkLA, walkRA)
print("-----")

animateSequence(initLX, initLY, initLZ, initRX, initRY, initRZ, walkLX, walkLY, walkLZ, walkRX, walkRY, walkRZ)

Initial backwards walk sequence:
0.0, -90.0, -2.3, -49.3, 31.2, -2.3, 2.3, 49.3, -31.2, 2.3, 90.0,
2.6, -84.8, -9.5, -58.0, 41.1, -9.5, -3.2, 46.0, -25.3, -3.2, 95.2,
5.0, -80.0, -16.4, -62.2, 48.1, -16.4, -6.9, 42.8, -19.8, -6.9, 100.0,
7.1, -75.9, -19.4, -61.7, 52.2, -19.4, -8.1, 42.0, -16.1, -8.1, 104.1,
8.7, -72.7, -16.4, -57.4, 53.4, -16.4, -6.9, 44.8, -15.2, -6.9, 107.3,
9.7, -70.7, -9.5, -49.7, 50.6, -9.5, -3.2, 50.3, -16.1, -3.2, 109.3,
-----
Continuous walk sequence:
0.0, -90.0, -19.4, -68.7, 43.7, -19.4, -8.1, 38.7, -22.6, -8.1, 90.0,
5.0, -80.0, -16.4, -62.2, 48.1, -16.4, -6.9, 42.8, -19.8, -6.9, 100.0,
8.7, -72.7, -9.5, -51.9, 48.3, -9.5, -3.2, 49.4, -18.5, -3.2, 107.3,
10.0, -70.0, -2.3, -38.8, 43.3, -2.3, 2.3, 56.6, -17.0, 2.3, 110.0,
8.7, -72.7, 3.2, -38.3, 35.5, 3.2, 9.5, 66.3, -28.3, 9.5, 107.3,
5.0, -80.0, 6.9, -37.6, 28.0, 6.9, 16.4, 70.4, -36.9, 16.4, 100.0,
0.0, -90.0, 8.1, -38.7, 22.6, 8.1, 19.4, 68.7, -43.7, 19.4, 90.0,
-5.0, -100.0, 6.9, -42.8, 19.8, 6.9, 16.4, 

# Turn in Place

In [12]:
x0 = 15
z0 = 125
dZ = 20
y0 = 5
dT = 10

zSteps = [z0, z0 - dZ*sin(pi/6), z0 - dZ*sin(pi/3), z0 - dZ, z0 - dZ*sin(pi/3), z0 - dZ*sin(pi/6)]
tSteps = [-dT, -dT, -dT, dT, dT, dT] # turn ccw


for i in range(6):
    rth1, rth2, rth3 = inverseKinematics(x0, y0, zSteps[i])
    rth1 = -1 * round(float(rth1 * 180 / pi), 1)
    rth2 = -1 * round(90-float(rth2 * 180 / pi), 1)
    rth3 = round(90-float(rth3 * 180 / pi), 1)
    
    lth1 = -1 * rth1
    lth2 = -1 * rth2
    lth3 = -1 * rth3
    tso = round(float(tSteps[i]), 1)
    
    print( str(tso) + ", " + 
        str(rth1) + ", " + str(rth2) + ", " + str(rth3) + ", " + str(rth1) + ", "
        + str(lth1) + ", " + str(lth2) + ", " + str(lth3) + ", " + str(lth1) + ", ")

-10.0, -2.3, -47.5, 33.8, -2.3, 2.3, 47.5, -33.8, 2.3, 
-10.0, -2.5, -53.1, 38.2, -2.5, 2.5, 53.1, -38.2, 2.5, 
-10.0, -2.7, -57.0, 41.1, -2.7, 2.7, 57.0, -41.1, 2.7, 
10.0, -2.7, -58.4, 42.1, -2.7, 2.7, 58.4, -42.1, 2.7, 
10.0, -2.7, -57.0, 41.1, -2.7, 2.7, 57.0, -41.1, 2.7, 
10.0, -2.5, -53.1, 38.2, -2.5, 2.5, 53.1, -38.2, 2.5, 


# Strafe / Crabwalk

Strategy: legs slide outwards, direction of strafe determined by which leg is pushing vertically

In [19]:
# Strafe using polar coords
x0 = 10
R0 = 125
dR = 30
th0 = 3.0
dTh = 15.0

thSteps = [th0, th0 + dTh*sin(pi/6), th0 + dTh*sin(pi/3), th0 + dTh, th0 + dTh*sin(pi/3), th0 + dTh*sin(pi/6)]
rSteps = [R0, R0 + dR*sin(pi/6), R0 + dR*sin(pi/3), R0 + dR, R0 + dR*sin(pi/3), R0 + dR*sin(pi/6)]

for i in range(6):
    th1, th2, th3 = inverseKPolar(rSteps[i], thSteps[i], x0)
    th1 = -1 * round(float(th1 * 180 / pi), 1)
    th2 = -1 * round(90-float(th2 * 180 / pi), 1)
    th3 = round(90-float(th3 * 180 / pi), 1)
    print(str(th1) + ", " + str(th2) + ", " + str(th3) + ", " + str(th1) + ", ") # pushing leg sequence (right leg)

    th1, th2, th3 = inverseKPolar(R0, thSteps[i], x0)
    th1 = round(float(th1 * 180 / pi), 1)
    th2 = round(90-float(th2 * 180 / pi), 1)
    th3 = -1 * round(90-float(th3 * 180 / pi), 1)
    print(str(th1) + ", " + str(th2) + ", " + str(th3) + ", " + str(th1) + ", ") # static leg sequence (left leg)

print("==============")

for i in range(6):
    th1, th2, th3 = inverseKPolar(R0, thSteps[i], x0)
    th1 = -1 * round(float(th1 * 180 / pi), 1)
    th2 = -1 * round(90-float(th2 * 180 / pi), 1)
    th3 = round(90-float(th3 * 180 / pi), 1)
    print(str(th1) + ", " + str(th2) + ", " + str(th3) + ", " + str(th1) + ", ") # static leg sequence (right leg)

    th1, th2, th3 = inverseKPolar(rSteps[i], thSteps[i], x0)
    th1 = round(float(th1 * 180 / pi), 1)
    th2 = round(90-float(th2 * 180 / pi), 1)
    th3 = -1 * round(90-float(th3 * 180 / pi), 1)
    print(str(th1) + ", " + str(th2) + ", " + str(th3) + ", " + str(th1) + ", ") # pushing leg sequence (left leg)

-3.0, -45.7, 36.6, -3.0, 
3.0, 45.7, -36.6, 3.0, 
-10.5, -36.6, 28.4, -10.5, 
10.5, 45.7, -36.6, 10.5, 
-16.0, -28.4, 20.8, -16.0, 
16.0, 45.7, -36.6, 16.0, 
-18.0, -24.7, 17.3, -18.0, 
18.0, 45.7, -36.6, 18.0, 
-16.0, -28.4, 20.8, -16.0, 
16.0, 45.7, -36.6, 16.0, 
-10.5, -36.6, 28.4, -10.5, 
10.5, 45.7, -36.6, 10.5, 
-3.0, -45.7, 36.6, -3.0, 
3.0, 45.7, -36.6, 3.0, 
-10.5, -45.7, 36.6, -10.5, 
10.5, 36.6, -28.4, 10.5, 
-16.0, -45.7, 36.6, -16.0, 
16.0, 28.4, -20.8, 16.0, 
-18.0, -45.7, 36.6, -18.0, 
18.0, 24.7, -17.3, 18.0, 
-16.0, -45.7, 36.6, -16.0, 
16.0, 28.4, -20.8, 16.0, 
-10.5, -45.7, 36.6, -10.5, 
10.5, 36.6, -28.4, 10.5, 


In [38]:
# punch pose
z0 = 120
x0 = 20
dX = 60
y0 = 20

# 30 -90
leftLeg = [x0 + 60, y0, z0]
# leftLeg = [x0, 0, 150]
rightLeg = [x0 - 90, y0, z0]

th1, th2, th3 = inverseKinematics(rightLeg[0], rightLeg[1], rightLeg[2])
th1 = -1*round(float(th1 * 180 / pi), 1)
th2 = -1*round(90-float(th2 * 180 / pi), 1)
th3 = round(90-float(th3 * 180 / pi), 1)
print(str(th1) + ", " + str(th2) + ", " + str(th3) + ", " + str(th1) + ", ")

th1, th2, th3 = inverseKinematics(leftLeg[0], leftLeg[1], leftLeg[2])
th1 = round(float(th1 * 180 / pi), 1)
th2 = round(90-float(th2 * 180 / pi), 1)
th3 = -1*round(90-float(th3 * 180 / pi), 1)
print(str(th1) + ", " + str(th2) + ", " + str(th3) + ", " + str(th1) + ", ")    

# display in graphics window
win = GraphWin("Window",640,640)
win.setCoords(-120,-280,120,5)

pline1 = None
pline2 = None
pline3 = None
pline4 = None

line1, line2, ptL = drawLeg(leftLeg[0], leftLeg[1], leftLeg[2])
line3, line4, ptR = drawLeg(rightLeg[0], rightLeg[1], rightLeg[2])
line1.setOutline('red')
line2.setOutline('red')
line3.setOutline('blue')
line4.setOutline('blue')
line1.draw(win)
line2.draw(win)
line3.draw(win)
line4.draw(win)

win.getMouse()
win.close()


-9.5, -2.4, 62.2, -9.5, 
9.5, 62.0, 4.6, 9.5, 


In [13]:
x0 = 15
z0 = 125
y0 = 5
dY = 10
dA = 30

walkLY = [y0 - dY, y0 - dY*cos(pi/6), y0 - dY*0.5, y0, y0 + dY*0.5, y0 + dY*cos(pi/6), 
          y0 + dY, y0 + dY*cos(pi/6), y0 + dY*0.5, y0, y0 - dY*0.5, y0 - dY*cos(pi/6)]
walkRY = [y0 + dY, y0 + dY*cos(pi/6), y0 + dY*0.5, y0, y0 - dY*0.5, y0 - dY*cos(pi/6), 
          y0 - dY, y0 - dY*cos(pi/6), y0 - dY*0.5, y0, y0 + dY*0.5, y0 + dY*cos(pi/6)]
armL = [0, -dA*sin(pi/12), -dA*sin(pi/6), -dA*sin(pi/4), -dA*sin(pi/3), -dA*sin(pi*5/12), 
        -dA, -dA*sin(pi*5/12), -dA*sin(pi/3), -dA*sin(pi/4), -dA*sin(pi/6), -dA*sin(pi/12)]
armR = [-dA, -dA*sin(pi*5/12), -dA*sin(pi/3), -dA*sin(pi/4), -dA*sin(pi/6), -dA*sin(pi/12),
        0, -dA*sin(pi/12), -dA*sin(pi/6), -dA*sin(pi/4), -dA*sin(pi/3), -dA*sin(pi*5/12)]

for i in range(len(walkLY)):
    rth1, rth2, rth3 = inverseKinematics(x0, walkRY[i], z0)
    rth1 = -1 * round(float(rth1 * 180 / pi), 1)
    rth2 = -1 * round(90-float(rth2 * 180 / pi), 1)
    rth3 = round(90-float(rth3 * 180 / pi), 1)

    lth1, lth2, lth3 = inverseKinematics(x0, walkLY[i], z0)
    lth1 = round(float(lth1 * 180 / pi), 1)
    lth2 = round(90-float(lth2 * 180 / pi), 1)
    lth3 = -1 * round(90-float(lth3 * 180 / pi), 1)

    ra = (round(float(armR[i]), 1))
    la = (round(-1*float(armL[i]), 1))
    
    print(
        str(ra) + ", " + str(rth1) + ", " + str(rth2) + ", " + str(rth3) + ", " + str(rth1) + ", "
        + str(lth1) + ", " + str(lth2) + ", " + str(lth3) + ", " + str(lth1) + ", " + str(la) + ",")

-60.0, -9.1, -46.6, 33.1, -9.1, -4.6, 47.3, -33.6, -4.6, -0.0,
-58.0, -8.2, -46.8, 33.2, -8.2, -3.7, 47.4, -33.7, -3.7, 15.5,
-52.0, -5.7, -47.2, 33.5, -5.7, -1.1, 47.5, -33.8, -1.1, 30.0,
-42.4, -2.3, -47.5, 33.8, -2.3, 2.3, 47.5, -33.8, 2.3, 42.4,
-30.0, 1.1, -47.5, 33.8, 1.1, 5.7, 47.2, -33.5, 5.7, 52.0,
-15.5, 3.7, -47.4, 33.7, 3.7, 8.2, 46.8, -33.2, 8.2, 58.0,
0.0, 4.6, -47.3, 33.6, 4.6, 9.1, 46.6, -33.1, 9.1, 60.0,
-15.5, 3.7, -47.4, 33.7, 3.7, 8.2, 46.8, -33.2, 8.2, 58.0,
-30.0, 1.1, -47.5, 33.8, 1.1, 5.7, 47.2, -33.5, 5.7, 52.0,
-42.4, -2.3, -47.5, 33.8, -2.3, 2.3, 47.5, -33.8, 2.3, 42.4,
-52.0, -5.7, -47.2, 33.5, -5.7, -1.1, 47.5, -33.8, -1.1, 30.0,
-58.0, -8.2, -46.8, 33.2, -8.2, -3.7, 47.4, -33.7, -3.7, 15.5,


In [41]:
x0 = 15
z0 = 125
dZ = 60
y0 = 5
dT = -15
dA = 10
dB = 30

arm_shoulder = [0, -dA*sin(pi/6), -dA*sin(pi/3), -dA, -dA*sin(pi/3), -dA*sin(pi/6), 
       0, dA*sin(pi/6), dA*sin(pi/3), dA, dA*sin(pi/3), dA*sin(pi/6)]
arm_bicep = [0, -dB*sin(pi/6), -dB*sin(pi/3), -dB, -dB*sin(pi/3), -dB*sin(pi/6), 
       0, dB*sin(pi/6), dB*sin(pi/3), dB, dB*sin(pi/3), dB*sin(pi/6)]
torso = [0, dT*sin(pi/6), dT*sin(pi/3), dT, dT*sin(pi/3), dT*sin(pi/6),
       0, -dT*sin(pi/6), -dT*sin(pi/3), -dT, -dT*sin(pi/3), -dT*sin(pi/6)]

for i in range(0,4):
    
    for j in range(len(arm)):
        tso = round(float(torso[j]), 1)
        la1 = round(90+float(arm_shoulder[j]), 1)
        la2 = round(float(arm_bicep[j]), 1)
        ra1 = round(-90+float(arm_shoulder[j]), 1)
        ra2 = round(float(arm_bicep[j]), 1)

        # rth1, rth2, rth3 = 0
        if i >= 2:
            rth1, rth2, rth3 = inverseKinematics(x0, y0, z0 - dZ + dZ*(i*12+j-24)/24)
        else:
            rth1, rth2, rth3 = inverseKinematics(x0, y0, z0 - dZ*(i*12+j)/24)
        rth1 = -round(float(rth1 * 180 / pi), 1)
        rth2 = -round(90-float(rth2 * 180 / pi), 1)
        rth3 = round(90-float(rth3 * 180 / pi), 1)
        lth1 = -rth1
        lth2 = -rth2        
        lth3 = -rth3

        print(
            str(tso) + ", " + str(ra1) + ", " + str(ra2) + ", "
            + str(rth1) + ", " + str(rth2) + ", " + str(rth3) + ", " + str(rth1) + ", "
            + str(lth1) + ", " + str(lth2) + ", " + str(lth3) + ", " + str(lth1) + ", " 
            + str(la1) + ", " + str(la2) + ", ")

0.0, -90.0, 0.0, -2.3, -47.5, 33.8, -2.3, 2.3, 47.5, -33.8, 2.3, 90.0, 0.0, 
-7.5, -95.0, -15.0, -2.3, -48.9, 34.9, -2.3, 2.3, 48.9, -34.9, 2.3, 85.0, -15.0, 
-13.0, -98.7, -26.0, -2.4, -50.3, 36.1, -2.4, 2.4, 50.3, -36.1, 2.4, 81.3, -26.0, 
-15.0, -100.0, -30.0, -2.4, -51.7, 37.2, -2.4, 2.4, 51.7, -37.2, 2.4, 80.0, -30.0, 
-13.0, -98.7, -26.0, -2.5, -53.1, 38.2, -2.5, 2.5, 53.1, -38.2, 2.5, 81.3, -26.0, 
-7.5, -95.0, -15.0, -2.5, -54.4, 39.2, -2.5, 2.5, 54.4, -39.2, 2.5, 85.0, -15.0, 
0.0, -90.0, 0.0, -2.6, -55.7, 40.2, -2.6, 2.6, 55.7, -40.2, 2.6, 90.0, 0.0, 
7.5, -85.0, 15.0, -2.7, -57.0, 41.2, -2.7, 2.7, 57.0, -41.2, 2.7, 95.0, 15.0, 
13.0, -81.3, 26.0, -2.7, -58.4, 42.1, -2.7, 2.7, 58.4, -42.1, 2.7, 98.7, 26.0, 
15.0, -80.0, 30.0, -2.8, -59.7, 43.0, -2.8, 2.8, 59.7, -43.0, 2.8, 100.0, 30.0, 
13.0, -81.3, 26.0, -2.9, -60.9, 43.9, -2.9, 2.9, 60.9, -43.9, 2.9, 98.7, 26.0, 
7.5, -85.0, 15.0, -2.9, -62.2, 44.8, -2.9, 2.9, 62.2, -44.8, 2.9, 95.0, 15.0, 
0.0, -90.0, 0.0, -3.0, -63.5, 45.

In [9]:
# gangnam style emote

### Leg motions
x0 = 15
z0 = 125
dZ = 30
y0 = 20
dY = 15

stepY = [0, dY*sin(pi/6), dY*sin(pi/3), dY, dY*sin(pi/3), dY*sin(pi/6),
        0, -dY*sin(pi/6), -dY*sin(pi/3), -dY, -dY*sin(pi/3), -dY*sin(pi/6),
        0, dY*sin(pi/6), dY*sin(pi/3), dY, dY*sin(pi/3), dY*sin(pi/6),
        0, dY*sin(pi/6), dY*sin(pi/3), dY, dY*sin(pi/3), dY*sin(pi/6),
        0, -dY*sin(pi/6), -dY*sin(pi/3), -dY, -dY*sin(pi/3), -dY*sin(pi/6),
         0, dY*sin(pi/6), dY*sin(pi/3), dY, dY*sin(pi/3), dY*sin(pi/6),
        0, -dY*sin(pi/6), -dY*sin(pi/3), -dY, -dY*sin(pi/3), -dY*sin(pi/6),
        0, -dY*sin(pi/6), -dY*sin(pi/3), -dY, -dY*sin(pi/3), -dY*sin(pi/6),]
stepZ = [z0, z0 + dZ*sin(pi/6), z0 + dZ*sin(pi/3), z0 + dZ, z0 + dZ*sin(pi/3), z0 + dZ*sin(pi/6)]

### Arm motions (1)
dA1_1 = 30
stepA1_1 = [-dA1_1, -dA1_1*sin(pi/3), -dA1_1*sin(pi/6), 0, -dA1_1*sin(pi/6), -dA1_1*sin(pi/3)]

### Arm motions (2)
dA3_2 = 30
dA4_2 = 30
stepA3_2 = [dA3_2, dA3_2*sin(pi/3),dA3_2*sin(pi/6),0,dA3_2*sin(pi/6),dA3_2*sin(pi/3)]
stepA4_2 = [dA4_2, dA4_2*sin(pi/3),dA4_2*sin(pi/6),0,dA4_2*sin(pi/6),dA4_2*sin(pi/3)]

elementCount = 0;
for i in range(len(stepY)):

    rth1, rth2, rth3 = inverseKinematics(x0, y0 + stepY[i], stepZ[i % 6])
    rth1 = -round(float(rth1 * 180 / pi), 1)
    rth2 = -round(90-float(rth2 * 180 / pi), 1)
    rth3 = round(90-float(rth3 * 180 / pi), 1)

    lth1, lth2, lth3 = inverseKinematics(x0, y0 - stepY[i], stepZ[i % 6])
    lth1 = round(float(lth1 * 180 / pi), 1)
    lth2 = round(90-float(lth2 * 180 / pi), 1)
    lth3 = -round(90-float(lth3 * 180 / pi), 1)

    ra1 = round(float(stepA1_1[i % 6]), 1)
    la1 = -ra1

    elementCount += printStep([ra1, rth1, rth2, rth3, rth1, lth1, lth2, lth3, lth1, la1])

print(elementCount)
print("=====")

elementCount = 0;
for i in range(len(stepY)):

    rth1, rth2, rth3 = inverseKinematics(x0, y0 + stepY[i], stepZ[i % 6])
    rth1 = -round(float(rth1 * 180 / pi), 1)
    rth2 = -round(90-float(rth2 * 180 / pi), 1)
    rth3 = round(90-float(rth3 * 180 / pi), 1)

    lth1, lth2, lth3 = inverseKinematics(x0, y0 - stepY[i], stepZ[i % 6])
    lth1 = round(float(lth1 * 180 / pi), 1)
    lth2 = round(90-float(lth2 * 180 / pi), 1)
    lth3 = -round(90-float(lth3 * 180 / pi), 1)

    ra3 = round(float(stepA3_2[i%6] - dA3_2*0.5 ), 1)
    ra4 = round(float(stepA4_2[i%6] - dA4_2*0.5 ), 1)

    elementCount += printStep([ra3, ra4, rth1, rth2, rth3, rth1, lth1, lth2, lth3, lth1])

print(elementCount)


-30.0,-9.1,-46.6,33.1,-9.1,9.1,46.6,-33.1,9.1,30.0,
-26.0,-11.1,-36.2,24.2,-11.1,5.1,37.7,-25.5,5.1,26.0,
-15.0,-12.3,-26.3,15.2,-12.3,2.7,29.5,-18.1,2.7,15.0,
0.0,-12.7,-21.3,10.6,-12.7,1.8,25.7,-14.7,1.8,-0.0,
-15.0,-12.3,-26.3,15.2,-12.3,2.7,29.5,-18.1,2.7,15.0,
-26.0,-11.1,-36.2,24.2,-11.1,5.1,37.7,-25.5,5.1,26.0,
-30.0,-9.1,-46.6,33.1,-9.1,9.1,46.6,-33.1,9.1,30.0,
-26.0,-5.1,-37.7,25.5,-5.1,11.1,36.2,-24.2,11.1,26.0,
-15.0,-2.7,-29.5,18.1,-2.7,12.3,26.3,-15.2,12.3,15.0,
0.0,-1.8,-25.7,14.7,-1.8,12.7,21.3,-10.6,12.7,-0.0,
-15.0,-2.7,-29.5,18.1,-2.7,12.3,26.3,-15.2,12.3,15.0,
-26.0,-5.1,-37.7,25.5,-5.1,11.1,36.2,-24.2,11.1,26.0,
-30.0,-9.1,-46.6,33.1,-9.1,9.1,46.6,-33.1,9.1,30.0,
-26.0,-11.1,-36.2,24.2,-11.1,5.1,37.7,-25.5,5.1,26.0,
-15.0,-12.3,-26.3,15.2,-12.3,2.7,29.5,-18.1,2.7,15.0,
0.0,-12.7,-21.3,10.6,-12.7,1.8,25.7,-14.7,1.8,-0.0,
-15.0,-12.3,-26.3,15.2,-12.3,2.7,29.5,-18.1,2.7,15.0,
-26.0,-11.1,-36.2,24.2,-11.1,5.1,37.7,-25.5,5.1,26.0,
-30.0,-9.1,-46.6,33.1,-9.1,9.1,46.6,-33.